In [1]:
%cd /scratch_net/biwidl214/ecetin_scratch/GSCodec

/scratch_net/biwidl214/ecetin_scratch/GSCodec


/scratch_net/biwidl214/ecetin/conda_envs/gscodec/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from models.splatting.hierarchical_model.hierarhical_utils import generate_octree
from training.base_gaussian_trainer import BaseGaussianTrainer
from config.build_config_spaces import ConfigReader
import torch

Using /home/ecetin/.cache/torch_extensions/py310_cu118 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/ecetin/.cache/torch_extensions/py310_cu118/pointops/build.ninja...
Building extension module pointops...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.


Loading extension module pointops...
Using /home/ecetin/.cache/torch_extensions/py310_cu118 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/ecetin/.cache/torch_extensions/py310_cu118/octree_generation/build.ninja...
Building extension module octree_generation...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module octree_generation...


ninja: no work to do.


Using /home/ecetin/.cache/torch_extensions/py310_cu118 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/ecetin/.cache/torch_extensions/py310_cu118/diff_gaussian_rasterization/build.ninja...
Building extension module diff_gaussian_rasterization...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.


Loading extension module diff_gaussian_rasterization...
Using /home/ecetin/.cache/torch_extensions/py310_cu118 as PyTorch extensions root...
No modifications detected for re-loaded extension module diff_gaussian_rasterization, skipping build step...
Loading extension module diff_gaussian_rasterization...


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Loaded compiled 3D CUDA chamfer distance


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: pug. Use `wandb login --relogin` to force relogin


In [3]:
reader = ConfigReader("config/preset_configs/hierarchical_gaussian.yaml")
dataset = reader.dataset_config
pipeline = reader.pipeline_config
optimization = reader.optimization_config

In [4]:
from models.splatting.hierarchical_model.hierarchical_gs import HierarchicalGSModel

In [5]:
dataset.data_path = dataset.data_path + "/tandt/train"
print(dataset.data_path)

/scratch_net/biwidl214/ecetin_scratch/data/tandt/train


In [6]:
gaussians = HierarchicalGSModel(dataset=dataset)
trainer = BaseGaussianTrainer(
    dataset, optimization, pipeline, gaussians, logger=None, checkpoint=None
)

Reading camera 301/301


In [15]:
class Octree:
    def __init__(self, gaussians, max_depth):
        # 3-sigma Gaussian ellipsoid surface approximation
        self.max_depth = max_depth
        self.gaussians = gaussians
        _, gaussian_node_assignments = generate_octree(
            points=gaussians._xyz, max_depth=max_depth
        )
        # self.assignments = gaussian_node_assignments

        unique_nodes, inverse_indices = torch.unique(
            gaussian_node_assignments, return_inverse=True)
        self.raw_unique_node_ids = unique_nodes
        self.inverse_indices = inverse_indices

        node_ids = torch.arange(0, unique_nodes.size(0), device=gaussian_node_assignments.device)
        
        # Map node ids to dense structure instead of sparse one
        self.processed_node_assignments = node_ids[inverse_indices]
        self.num_unique_nodes = unique_nodes.size(0)

In [16]:
octree = Octree(gaussians, 10)

In [20]:
print(octree.processed_node_assignments)
print(octree.num_unique_nodes)

tensor([[    2,    14,    51,  ...,  5826, 11061, 19764],
        [    2,    14,    51,  ...,  5828, 11077, 19861],
        [    2,    14,    51,  ...,  5814, 10979, 19418],
        ...,
        [    2,    14,    51,  ...,  5814, 10976, 19394],
        [    2,    14,    51,  ...,  5814, 10982, 19437],
        [    2,    14,    51,  ...,  5828, 11079, 19878]], device='cuda:0')
25307


In [21]:
octree.gaussians.get_covariance_matrix().shape

torch.Size([182686, 3, 3])

In [14]:
import torch

# Example tensors
N = 100  # Number of base Gaussians
depth_levels = 10
means = torch.randn(N, 3)  # Random means for base level
variances = torch.randn(N, 3, 3)  # Random variances for base level
variances = torch.matmul(variances, variances.transpose(-1, -2))  # Ensure positive semi-definite
node_ids = torch.randint(0, 1000, (N, depth_levels))  # Random node IDs for octree placement

# Uniform weights initialization (customize as necessary)
weights = torch.ones(N) / N

# Containers for all levels
all_means = [means]
all_variances = [variances]
all_weights = [weights]

# Process each level from bottom to top
for level in range(depth_levels - 1, 0, -1):
    new_means = []
    new_variances = []
    new_weights = []

    unique_nodes = torch.unique(node_ids[:, level])
    for node in unique_nodes:
        mask = node_ids[:, level] == node
        current_means = all_means[-1][mask]
        current_variances = all_variances[-1][mask]
        current_weights = all_weights[-1][mask]

        total_weight = current_weights.sum()
        if total_weight == 0:
            continue

        # Weighted sum of means
        weighted_mean = torch.sum(current_weights[:, None] * current_means, dim=0) / total_weight

        # Calculate weighted covariance
        mean_diff = current_means - weighted_mean
        weighted_covariance = torch.sum(current_weights[:, None, None] * (current_variances + torch.bmm(mean_diff[:, :, None], mean_diff[:, None, :])), dim=0) / total_weight

        new_means.append(weighted_mean)
        new_variances.append(weighted_covariance)
        new_weights.append(total_weight)

    # Prepare for next level
    all_means.append(torch.stack(new_means))
    all_variances.append(torch.stack(new_variances))
    all_weights.append(torch.tensor(new_weights))

# Final means, variances, and weights for top level
final_means = all_means[-1]
final_variances = all_variances[-1]
final_weights = all_weights[-1]


IndexError: The shape of the mask [100] at index 0 does not match the shape of the indexed tensor [93, 3] at index 0

In [13]:
depth_levels = octree.max_depth
processed_node_assignments = octree.processed_node_assignments
xyzs = octree.gaussians._xyz
variances = octree.gaussians.get_covariance()

# Process each level from bottom to top
for level in range(depth_levels - 1, 0, -1):
    unique_nodes = torch.unique(processed_node_assignments[:, level])
    new_means = torch.zeros(len(unique_nodes), 3)
    new_variances = torch.zeros(len(unique_nodes), 3, 3)

    for node in unique_nodes:
        mask = processed_node_assignments[:, level] == node
        current_xyzs = xyzs[mask]
        current_variances = final_variances[mask]
        current_weights = final_weights[mask]

        total_weight = current_weights.sum()
        if total_weight == 0:
            continue

        # Weighted sum of means
        weighted_mean = torch.sum(current_weights[:, None] * current_means, dim=0) / total_weight

        # Calculate weighted covariance
        mean_diff = current_means - weighted_mean
        weighted_covariance = torch.sum(current_weights[:, None, None] * (current_variances + torch.bmm(mean_diff[:, :, None], mean_diff[:, None, :])), dim=0) / total_weight

        new_means[node] = weighted_mean
        new_variances[node] = weighted_covariance
    

{13: tensor([0.3826, 1.6565, 0.8319]),
 34: tensor([ 1.3779, -1.2833,  0.0787]),
 52: tensor([ 0.6757,  1.4601, -0.1231]),
 66: tensor([-0.0626,  0.1199,  0.9467]),
 99: tensor([ 1.3779, -1.2833,  0.0787]),
 100: tensor([-0.0323, -0.5251, -1.2345]),
 107: tensor([ 2.5830, -0.4467, -0.0721]),
 111: tensor([-0.8225,  0.0525,  1.5812]),
 126: tensor([ 0.3185,  0.1256, -0.1990]),
 140: tensor([ 1.0446, -0.0795, -0.4732]),
 146: tensor([ 2.5830, -0.4467, -0.0721]),
 160: tensor([-1.0644,  0.2716, -1.3847]),
 171: tensor([ 0.8802, -0.3103, -1.1658]),
 180: tensor([-1.5868,  0.2041, -1.0191]),
 181: tensor([-1.3276e+00, -1.3072e+00, -3.7756e-04]),
 183: tensor([-0.5879,  1.1570,  1.1342]),
 185: tensor([-0.6569, -0.1301,  2.1359]),
 212: tensor([-0.1520, -1.4269, -0.4933]),
 213: tensor([ 0.1730,  0.4658, -2.0752]),
 220: tensor([ 3.3826, -0.5855, -1.6703]),
 224: tensor([ 0.2096, -1.8355,  0.8107]),
 227: tensor([ 0.2096, -1.8355,  0.8107]),
 250: tensor([0.3695, 0.2487, 0.1080]),
 269: tens